In [17]:
import os

import scanpy as sc
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from timm import create_model
from einops import rearrange, reduce

In [2]:
from deep_spatial_genomics.models.x_unet import XUnet

#### test data

In [3]:
a = sc.read_h5ad('/storage1/fs1/dinglab/Active/Projects/estorrs/deep-spatial-genomics/data/brca/HT397B1-S1H2A4/adata.h5ad')
a

AnnData object with n_obs × n_vars = 3561 × 17943
    obs: 'in_tissue', 'array_row', 'array_col', 'clusters', 'alla_label'
    var: 'gene_ids', 'feature_types', 'genome', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'alla_label_colors', 'full_res_he', 'hvg', 'leiden', 'log1p', 'neighbors', 'pca', 'spatial', 'umap'
    obsm: 'X_pca', 'X_umap', 'spatial'
    varm: 'PCs'
    layers: 'counts'
    obsp: 'connectivities', 'distances'

#### model

In [ ]:
"""
modified from https://gist.github.com/rwightman/f8b24f4e6f5504aba03e999e02460d31
"""
class Unet(nn.Module):
    """Unet is a fully convolution neural network for image semantic segmentation
    Args:
        encoder_name: name of classification model (without last dense layers) used as feature
            extractor to build segmentation model.
        encoder_weights: one of ``None`` (random initialization), ``imagenet`` (pre-training on ImageNet).
        decoder_channels: list of numbers of ``Conv2D`` layer filters in decoder blocks
        decoder_use_batchnorm: if ``True``, ``BatchNormalisation`` layer between ``Conv2D`` and ``Activation`` layers
            is used.
        num_classes: a number of classes for output (output shape - ``(batch, classes, h, w)``).
        center: if ``True`` add ``Conv2dReLU`` block on encoder head
    NOTE: This is based off an old version of Unet in https://github.com/qubvel/segmentation_models.pytorch
    """

    def __init__(
            self,
            backbone='resnet50',
            backbone_kwargs=None,
            backbone_indices=None,
            decoder_use_batchnorm=True,
            decoder_channels=(256, 128, 64, 32, 16),
            in_chans=1,
            num_classes=5,
            center=False,
            norm_layer=nn.BatchNorm2d,
    ):
        super().__init__()
        backbone_kwargs = backbone_kwargs or {}
        # NOTE some models need different backbone indices specified based on the alignment of features
        # and some models won't have a full enough range of feature strides to work properly.
        encoder = create_model(
            backbone, features_only=True, out_indices=backbone_indices, in_chans=in_chans,
            pretrained=True, **backbone_kwargs)
        encoder_channels = encoder.feature_info.channels()[::-1]
        self.encoder = encoder

        if not decoder_use_batchnorm:
            norm_layer = None
        self.decoder = UnetDecoder(
            encoder_channels=encoder_channels,
            decoder_channels=decoder_channels,
            final_channels=num_classes,
            norm_layer=norm_layer,
            center=center,
        )

    def forward(self, x: torch.Tensor):
        x = self.encoder(x)
        x.reverse()  # torchscript doesn't work with [::-1]
        x = self.decoder(x)
        return x


class Conv2dBnAct(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, padding=0,
                 stride=1, act_layer=nn.ReLU, norm_layer=nn.BatchNorm2d):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride=stride, padding=padding, bias=False)
        self.bn = norm_layer(out_channels)
        self.act = act_layer(inplace=True)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.act(x)
        return x


class DecoderBlock(nn.Module):
    def __init__(self, in_channels, out_channels, scale_factor=2.0, act_layer=nn.ReLU, norm_layer=nn.BatchNorm2d):
        super().__init__()
        conv_args = dict(kernel_size=3, padding=1, act_layer=act_layer)
        self.scale_factor = scale_factor
        if norm_layer is None:
            self.conv1 = Conv2dBnAct(in_channels, out_channels, **conv_args)
            self.conv2 = Conv2dBnAct(out_channels, out_channels,  **conv_args)
        else:
            self.conv1 = Conv2dBnAct(in_channels, out_channels, norm_layer=norm_layer, **conv_args)
            self.conv2 = Conv2dBnAct(out_channels, out_channels, norm_layer=norm_layer, **conv_args)

    def forward(self, x, skip: Optional[torch.Tensor] = None):
        if self.scale_factor != 1.0:
            x = F.interpolate(x, scale_factor=self.scale_factor, mode='nearest')
        if skip is not None:
            x = torch.cat([x, skip], dim=1)
        x = self.conv1(x)
        x = self.conv2(x)
        return x


class UnetDecoder(nn.Module):

    def __init__(
            self,
            encoder_channels,
            decoder_channels=(256, 128, 64, 32, 16),
            final_channels=1,
            norm_layer=nn.BatchNorm2d,
            center=False,
    ):
        super().__init__()

        if center:
            channels = encoder_channels[0]
            self.center = DecoderBlock(channels, channels, scale_factor=1.0, norm_layer=norm_layer)
        else:
            self.center = nn.Identity()

        in_channels = [in_chs + skip_chs for in_chs, skip_chs in zip(
            [encoder_channels[0]] + list(decoder_channels[:-1]),
            list(encoder_channels[1:]) + [0])]
        out_channels = decoder_channels

        self.blocks = nn.ModuleList()
        for in_chs, out_chs in zip(in_channels, out_channels):
            self.blocks.append(DecoderBlock(in_chs, out_chs, norm_layer=norm_layer))
        self.final_conv = nn.Conv2d(out_channels[-1], final_channels, kernel_size=(1, 1))

        self._init_weight()

    def _init_weight(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                torch.nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def forward(self, x: List[torch.Tensor]):
        encoder_head = x[0]
        skips = x[1:]
        x = self.center(encoder_head)
        for i, b in enumerate(self.blocks):
            skip = skips[i] if i < len(skips) else None
            x = b(x, skip)
        x = self.final_conv(x)
        return x

In [ ]:
class xFuse(nn.Module):
    def __init__(self, n_genes, voxel_size, n_covariates,
                 n_metagenes=20, latent_dim=64, in_channels=3, decoder_channels=(64, 32, 16),
                 kl_scaler = .001, latent_scaler=1., L_scaler=1., E_scaler=1., F_scaler=1.):
        super().__init__()
        
        self.unet = Unet(backbone='resnet50',
                        decoder_channels=(64, 32, 16),
                        in_chans=in_channels,
                        num_classes=latent_dim)
        
        self.n_genes = n_genes
        self.voxel_size = voxel_size
        self.n_metagenes = n_metagenes
        self.latent_dim = latent_dim
        
        # scalers for KL loss on q for latent, L, E, and F
        self.latent_scaler = latent_scaler
        self.L_scaler = L_scaler
        self.E_scaler = E_scaler
        self.F_scaler = F_scaler
        self.kl_scaler = kl_scaler
        
        # gene-wise baselines for r and p
        self.t = nn.Parameter(torch.randn(self.n_genes))
        self.u = nn.Parameter(torch.randn(self.n_genes))
        
        # latent mu and var
        self.latent_mu = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=1)
        self.latent_var = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=1)
        
        # creates initial metagene matrix - (b, H, W, m)
        self.h = nn.Sequential(
            nn.Conv2d(in_channels=latent_dim, out_channels=self.n_metagenes, kernel_size=1),
            nn.Softmax(dim=-1)
        )
        
        # creates scale factor for each pixel - (1, H, W)
        self.s = nn.Sequential(
            nn.Conv2d(in_channels=latent_dim, out_channels=1, kernel_size=1),
            nn.Softplus()
        )
        
        # gene activity matrix - (m, g)
        self.L = nn.Parameter(torch.randn(self.n_metagenes, self.n_genes))
        self.L_mu = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=1)
        self.L_var = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=1)
        
        # E - (D, g) - covariate matrix for r
        self.E = nn.Parameter(torch.randn(self.n_covariates, self.n_genes))
        self.E_mu = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=1)
        self.E_var = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=1)
        
        # F - (D, g) - covariate matrix for p
        self.F = nn.Parameter(torch.randn(self.n_covariates, self.n_genes))
        self.F_mu = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=1)
        self.F_var = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=1)
        
        self.reconstruction_loss = nn.MSELoss()
        
    def _sample_q(self, mu, log_var, use_means=False):
        # sample z from parameterized distributions
        std = torch.exp(log_var / 2)
        q = torch.distributions.Normal(mu, std)
        # get our latent
        if use_means:
            z = mu
        else:
            z = q.rsample()

        return z, mu, std
    
    def _kl_divergence(self, z, mu, std):
        # lightning imp.
        # Monte carlo KL divergence
        p = torch.distributions.Normal(torch.zeros_like(mu), torch.ones_like(std))
        q = torch.distributions.Normal(mu, std)

        log_qzx = q.log_prob(z)
        log_pz = p.log_prob(z)

        kl = (log_qzx - log_pz)
        kl = kl.sum(-1)

        return kl
    
     def calculate_loss(self, x_raw, result):
        nb = torch.distributions.NegativeBinomial(result['r'], result['p'])
        reconstruction_loss = torch.mean(-nb.log_prob(x_raw))
        
        latent_loss = torch.mean(self.kl_divergence(result['z'], result['z_mu'], result['z_std']))
        L_loss = torch.mean(self.kl_divergence(result['L'], result['L_mu'], result['L_std']))
        E_loss = torch.mean(self.kl_divergence(result['E'], result['E_mu'], result['E_std']))
        F_loss = torch.mean(self.kl_divergence(result['F'], result['F_mu'], result['F_std']))

        kl_loss = (
            latent_loss * self.latent_scaler,
            L_loss * self.L_scaler,
            E_loss * self.E_scaler,
            F_loss * self.F_scaler
        )

        return {
            'overall_loss': reconstruction_loss + self.kl_scaler * kl_loss,
            'reconstruction_loss': reconstruction_loss,
            'kl_loss': kl_loss,
            'latent_loss': latent_loss,
            'L_loss': L_loss,
            'E_loss': E_loss,
            'F_loss': F_loss,
        }
        
    def encode(self, x, use_means=False):
        x_encoded = self.unet(x)
        
        mu, log_var = self.latent_mu(x_encoded), self.latent_var(x_encoded)

        return self._sample_q(mu, log_var, use_means=use_means), mu, log_var
        
    def sample_L(self, use_means=False):
        mu, log_var = self.L_mu(self.L), self.L_var(self.L)

        return self._sample_q(mu, log_var, use_means=use_means), mu, log_var
    
    def sample_E(self, use_means=False):
        mu, log_var = self.E_mu(self.E), self.E_var(self.E)

        return self._sample_q(mu, log_var, use_means=use_means), mu, log_var
    
    def sample_F(self, use_means=False):
        mu, log_var = self.F_mu(self.F), self.F_var(self.F)

        return self._sample_q(mu, log_var, use_means=use_means), mu, log_var
        
    def metagenes_from_latent(self, z):
        h = self.h(z) # (b, m, H, W)
        s = self.s(z) # (1, H, W, 1)
        return rearrange(h * s, 'b c h w -> b h w c')
    
    def parameterize_nb(self, z, b, voxel_level=True, use_means=False, return_intermediates=False):
        """
        reconstruct expression from latent z and covariate vector b
        
        z - (b, H, W, latent_dim)
        b - (b, D)
        """
        h = self.metagenes_from_latent(z) # (b, H, W, m)
        
        if voxel_level:
            # aggregate and sum pixels to the voxel level where Hv and Wv are height and width in terms of voxels
            h = reduce(h, 'b (h h2) (w w2) c -> b h w c', 'sum',
                       h2=voxel_size, w2=voxel_size) # (b, Hv, Wv, m)
        
        # calculate r for each gene
        L, L_mu, L_std = self.sample_L(use_means=use_means) # (m, g)
        r = h@L # (b, Hv, Wv, g)
        
        # adjust for covariates
        E, E_mu, E_std = self.sample_E(use_means=use_means) # (D, g)
        adj = torch.exp(b@E + self.t) # (1, g)
        r *= adj # (b, Hv, Wv, g)
        
        # calculate p
        F, F_mu, F_std = self.sample_F(use_means=use_means)
        p = torch.sigmoid(b@F + self.u) # (b, g)
        
        if return_intermediates:
            return {
                'L': L,
                'L_mu': L_mu,
                'L_std': L_std,
                'E': E,
                'E_mu': E_mu,
                'E_std': E_std,
                'F': F,
                'F_mu': F_mu,
                'F_std': F_std,
                'metagene_matrix': h,
                'r': r,
                'p': p
            }
        
        return r, p
    
    def forward_with_expression(self, x, b, use_means=False, voxel_level=True):
        result = self.forward(x, b, voxel_level=voxel_level,
                              use_means=use_means, return_intermediates=True)
        r, p = result['r'], result['p']
        
        # get expression from negative binomial
        nb = torch.distributions.NegativeBinomial(r, p)
        result['expression'] = nb.mean()
                
        return result
    
    def forward(self, x, b, voxel_level=True, use_means=False, return_intermediates=True):
        z, z_mu, z_std = self.encode(x, use_means=use_means)
        result = self.parameterize_nb(z, b, voxel_level=voxel_level,
                                      use_means=use_means, return_intermediates=return_intermediates)
        result.update({
            'z': z,
            'z_mu': z_mu,
            'z_std': z_std
        })
        
        return result
        
    
    

In [60]:
r = F.softplus(torch.randn(1, 5, 5, 100))
p = torch.sigmoid(torch.randn(1, 100))

nb = torch.distributions.NegativeBinomial(r, p)
s = nb.sample()
log_prob = -nb.log_prob(s)
log_prob.shape



torch.Size([1, 5, 5, 100])

In [56]:
nb.mean

tensor([ 10.0000, 180.0000,   1.0000])

In [58]:
r = torch.Tensor([10., 20., 1.])
p = torch.Tensor([.5, .9, .5])

nb = torch.distributions.NegativeBinomial(r, p)
s = nb.sample()
log_prob = -nb.log_prob(s)
s, log_prob



(tensor([ 15., 208.,   2.]), tensor([3.2450, 5.0071, 2.0794]))

In [50]:
a = torch.randn(1, 50, 50, 20)
reduce(a, 'b (h h2) (w w2) c -> b h w c', 'sum', h2=10, w2=10).shape

torch.Size([1, 5, 5, 20])

In [22]:
a = torch.randn(1, 5, 5, 20)
b = torch.randn(20, 100)
(a@b).shape

torch.Size([1, 5, 5, 100])

In [23]:
a = torch.randn(1, 50, 50, 20)
b = torch.randn(1, 20)
(a * b).shape

torch.Size([1, 50, 50, 20])

In [ ]:
class PollockModel(torch.nn.Module):
    def __init__(self, genes, classes,
                 latent_dim=64, enc_out_dim=128, middle_dim=512,
                 zinb_scaler=1., kl_scaler=1e-5, clf_scaler=1.):
        """
        Pollock VAE + classifier
        """
        super(PollockModel, self).__init__()
        self.latent_dim = latent_dim
        self.genes = genes
        self.n_genes = len(genes)
        self.classes = classes
        self.n_classes = len(classes)

        self.encoder = torch.nn.Sequential(
            torch.nn.Linear(self.n_genes, middle_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(middle_dim, enc_out_dim),
            torch.nn.ReLU(),
        )

        self.mu = torch.nn.Linear(enc_out_dim, latent_dim)
        self.var = torch.nn.Linear(enc_out_dim, latent_dim)

        self.decoder = torch.nn.Sequential(
            torch.nn.Linear(latent_dim, enc_out_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(enc_out_dim, middle_dim),
            torch.nn.ReLU(),
        )
        self.disp_decoder = torch.nn.Sequential(
            torch.nn.Linear(middle_dim, self.n_genes),
            DispAct()
        )
        self.mean_decoder = torch.nn.Sequential(
            torch.nn.Linear(middle_dim, self.n_genes),
            MeanAct()
        )
        self.drop_decoder = torch.nn.Sequential(
            torch.nn.Linear(middle_dim, self.n_genes),
            torch.nn.Sigmoid()
        )

        self.prediction_head = torch.nn.Sequential(
            torch.nn.Linear(latent_dim, latent_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(latent_dim, self.n_classes),
            torch.nn.Softmax(dim=1),
        )

        self.zinb_loss = ZINBLoss()
        self.ce_loss = torch.nn.CrossEntropyLoss()
        self.zinb_scaler = zinb_scaler
        self.kl_scaler = kl_scaler
        self.clf_scaler = clf_scaler

    def kl_divergence(self, z, mu, std):
        # lightning imp.
        # Monte carlo KL divergence
        p = torch.distributions.Normal(torch.zeros_like(mu), torch.ones_like(std))
        q = torch.distributions.Normal(mu, std)

        log_qzx = q.log_prob(z)
        log_pz = p.log_prob(z)

        kl = (log_qzx - log_pz)
        kl = kl.sum(-1)

        return kl

    def encode(self, x, use_means=False):
        x_encoded = self.encoder(x)
        mu, log_var = self.mu(x_encoded), self.var(x_encoded)

        # sample z from parameterized distributions
        std = torch.exp(log_var / 2)
        q = torch.distributions.Normal(mu, std)
        # get our latent
        if use_means:
            z = mu
        else:
            z = q.rsample()

        return z, mu, std

    def decode(self, x):
        h = self.decoder(x)
        x_disp = self.disp_decoder(h)
        x_mean = self.mean_decoder(h)
        x_drop = self.drop_decoder(h)

        return x_disp, x_mean, x_drop

    def calculate_loss(self, r, x_raw, scale_factor, y_true):
        reconstruction_loss = self.zinb_loss(
            x_raw, r['x_mean'], r['x_disp'], r['x_drop'], scale_factor=scale_factor)

        kl_loss = torch.mean(self.kl_divergence(r['z'], r['mu'], r['std']))

        clf_loss = torch.mean(self.ce_loss(r['y'], y_true))

        return ((reconstruction_loss * self.zinb_scaler) + (kl_loss * self.kl_scaler) + (clf_loss * self.clf_scaler),
                reconstruction_loss,
                kl_loss,
                clf_loss)

    def forward(self, x, use_means=False):
        z, mu, std = self.encode(x, use_means=use_means)
        x_disp, x_mean, x_drop = self.decode(z)
        y = self.prediction_head(z)

        return {
            'z': z,
            'mu': mu,
            'std': std,
            'x_disp': x_disp,
            'x_mean': x_mean,
            'x_drop': x_drop,
            'y': y
        }